# First part

## Wikipedia Scraping

In [107]:
import urllib.request

In [108]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [109]:
page = urllib.request.urlopen(url)

In [110]:
from bs4 import BeautifulSoup

In [111]:
soup = BeautifulSoup(page, "lxml")

In [112]:
right_table=soup.find('table', class_='wikitable sortable')

In [113]:
A=[]
B=[]
C=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

## Pandas processing

In [114]:
import pandas as pd

In [135]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighbourhood']=C
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Extracting the rows with Borough not assigned :

In [168]:
right_df = df[df['Borough']!='Not assigned\n']
right_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Checking if there is any Neighbourhood not assigned left :

In [137]:
right_df[right_df['Neighbourhood']=='Not assigned\n']

,Postal Code,Borough,Neighbourhood


There isn't ! Our dataframe is almost ready.

Just a removal of unnecessary "\n" at the end of each line, and there we go :

In [170]:
right_df = right_df.replace('\n','', regex=True)

In [139]:
right_df.shape

(103, 3)

# Second part

In [140]:
!pip install geocoder

In [141]:
import geocoder

In [177]:
latitude=[]
longitude=[]

for n in range(len(right_df)):
    
    postal_code = right_df.iloc[n,0]

    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    
right_df['Latitude'] = latitude
right_df['Longitude'] = longitude

Our dataframe is ready :

In [181]:
right_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.75188,-79.33036
3,M4A,North York,Victoria Village,43.73042,-79.31282
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302


# Third part

Let's extract Toronto boroughs from the original dataframe :

In [198]:
toronto_df = right_df[right_df['Borough'].str.find("Toronto")>-1]